To run this notebook, start Jupyter as follows from the Django project root: <br>
(https://medium.com/ayuth/how-to-use-django-in-jupyter-notebook-561ea2401852)

`python manage.py shell_plus --notebook`

<br>
NOTE:  You will need to changed the kernel from menu: `Kernel` > `Change kernel` > `Django Shell-Plus`

In [1]:
import os
import logging
import pandas as pd
import geopandas as gpd
import datetime

# for census data used in Bubble maps
from census import Census

# import variables in the local .env file
from dotenv import load_dotenv
load_dotenv(verbose=False)

True

In [21]:
# avoid chained assignment warnings
pd.options.mode.chained_assignment = None

# show all rows and columns
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

In [3]:
!pwd

/Users/markmcdonald/Desktop/marks-covid-tracker/tracker/notebooks


In [4]:
# GLOBAL VARIABLES
FILE_PATH = os.path.join('..', 'covid_tracker', 'COVID-19', 'csse_covid_19_data', 'csse_covid_19_time_series')

territories = ['American Samoa', 'Guam', 'Northern Mariana Islands', 'Mariana Islands',
               'Puerto Rico', 'Virgin Islands', 'Diamond Princess', 'Grand Princess']

# affiliation of governor in 2019
political_affiliations = {'American Samoa': 'na',
                          'Guam': 'na',
                          'Northern Mariana Islands': 'na',
                          'Puerto Rico': 'na',
                          'Virgin Islands': 'na',
                          'Alabama': 'red',
                          'Alaska': 'red',
                          'Arizona': 'red',
                          'Arkansas': 'red',
                          'California': 'blue',
                          'Colorado': 'blue',
                          'Connecticut': 'blue',
                          'Delaware': 'blue',
                          'District of Columbia': 'blue',
                          'Florida': 'red',
                          'Georgia': 'red',
                          'Hawaii': 'blue',
                          'Idaho': 'red',
                          'Illinois': 'blue',
                          'Indiana': 'red',
                          'Iowa': 'red',
                          'Kansas': 'blue',
                          'Kentucky': 'blue',
                          'Louisiana': 'blue',
                          'Maine': 'blue',
                          'Maryland': 'red',
                          'Massachusetts': 'red',
                          'Michigan': 'blue',
                          'Minnesota': 'blue',
                          'Mississippi': 'red',
                          'Missouri': 'red',
                          'Montana': 'blue',
                          'Nebraska': 'red',
                          'Nevada': 'blue',
                          'New Hampshire': 'red',
                          'New Jersey': 'blue',
                          'New Mexico': 'blue',
                          'New York': 'blue',
                          'North Carolina': 'blue',
                          'North Dakota': 'red',
                          'Ohio': 'red',
                          'Oklahoma': 'red',
                          'Oregon': 'blue',
                          'Pennsylvania': 'blue',
                          'Rhode Island': 'blue',
                          'South Carolina': 'red',
                          'South Dakota': 'red',
                          'Tennessee': 'red',
                          'Texas': 'red',
                          'Utah': 'red',
                          'Vermont': 'red',
                          'Virginia': 'blue',
                          'Washington': 'blue',
                          'West Virginia': 'red',
                          'Wisconsin': 'blue',
                          'Wyoming': 'red',
                          'Diamond Princess': 'na',
                          'Grand Princess': 'na'}

In [5]:
try:
    from django.http import JsonResponse

except Exception as e:
    print("NOT LOADED:  start notebook with:\n")
    print("\tpython manage.py shell_plus --notebook")
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [6]:
!pwd

/Users/markmcdonald/Desktop/marks-covid-tracker/tracker/notebooks


In [7]:
import git
import json
from django.http import JsonResponse

def refresh_git(request):
    """
    Triggers a pull of the Git repository that hold the data used in plots.

    :param request: The HTML request which is provided by Django when the route is called.  No values of the request are used in this function.

    :return: A Json formatted text response that includes the git activity of the request to pull data.
    """
    try:
        g = git.cmd.Git('../covid_tracker/COVID-19')
        rv = g.pull()
    except Exception as e:
        rv = "Git sync in process..."

    print(rv)
#     return JsonResponse(json.dumps(rv), safe=False)

In [8]:
refresh_git(None)

Git sync in process...


In [9]:
# Get vaccination data
# Dataset web page
# https://data.cdc.gov/Vaccinations/COVID-19-Vaccinations-in-the-United-States-County/8xkx-amqh

# Download CSV Link
# https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD
# https://data.cdc.gov/api/views/8xkx-amqh/rows.csv?accessType=DOWNLOAD&api_foundry=true

In [10]:
## NEXT: 
## Save data into CSV file
## Preload from CSV file before using API to get data

In [11]:
FILE_PATH_VAXDATA = os.path.join('..', 'covid_tracker', 'data', 'vaccinations')
os.path.isdir(FILE_PATH_VAXDATA)

True

In [16]:
import warnings
warnings.filterwarnings("ignore")

from sodapy import Socrata
from us import states

FILE_PATH_VAXDATA = os.path.join('..', 'covid_tracker', 'data', 'vaccinations')

class Vax_Data:
    _json_cache: {} = {} # holds raw JSON data by date for reuse - avoids re-downloading for one data type
    _base_date: datetime.date = datetime.date(year=2020, month=12, day=13) # first date of available vaccination data 2020-12-13
    _vax_dfs = {
                    'series_complete_pop_pct': None,
                    'series_complete_yes': None,
                    'series_complete_12plus': None,
                    'series_complete_12pluspop': None,
                    'series_complete_18plus': None,
                    'series_complete_18pluspop': None,
                    'series_complete_65plus': None,
                    'series_complete_65pluspop': None,
                    'completeness_pct': None,
                    'administered_dose1_recip': None,
                    'administered_dose1_pop_pct': None,
                    'administered_dose1_recip_12plus': None,
                    'administered_dose1_recip_12pluspop_pct': None,
                    'administered_dose1_recip_18plus': None,
                    'administered_dose1_recip_18pluspop_pct': None,
                    'administered_dose1_recip_65plus': None,
                    'administered_dose1_recip_65pluspop_pct': None
                }
    _population_df: pd.DataFrame = None
    
    
    def __init__(self, verbose=False):
        # initialize df with basedate if file doesn't exist
        self._initialize_files()
        
        # initialize the dataframes from file data
        self._initialize_dataframes()
        
        # bring all the files up-to-date
        self._update_dataframes(verbose=verbose)
        
        # add the population data to reusable dataframe
        self._population_df = self._get_population_df()
        
    
    
    # create csv data file for each data category, if it doesn't already exist
    def _initialize_files(self) -> None:
        for _data_type, _vax_df in self._vax_dfs.items():
            # check to see if there is a CSV to start with
            if not os.path.isfile(os.path.join(FILE_PATH_VAXDATA, _data_type)):
                _df = self._get_dataframe_for_date(for_date=self._base_date, 
                                                   data_type=_data_type)
                    
                if _df is None:
                    print(f"No dataframe for '{_data_type}'")
                    continue
                
                _df.to_csv(os.path.join(FILE_PATH_VAXDATA, _data_type))
                print(f"Created file for: {str(self._base_date)} -> {_data_type}")
    
    def _initialize_dataframes(self) -> None:
        for _data_type, _vax_df in self._vax_dfs.items():
            _df = pd.read_csv(os.path.join(FILE_PATH_VAXDATA, _data_type))
            # pad FIPS code and set the index
            _df.fips = _df.fips.astype(str).str.pad(width=5, side='left', fillchar='0')
            _index_columns = ['fips', 'recip_county', 'recip_state']
            _df.set_index(_index_columns, inplace=True)

            self._vax_dfs.update({_data_type: _df})
    
    def _update_dataframes(self, verbose:bool) -> None:
        for _data_type, _vax_df in self._vax_dfs.items():
            self._update_dataframe(_data_type, verbose)
                        
        
    # get json data from cache or add data to cache if it doesn't exist
    def _get_json_data_for_date(self, for_date:datetime.date) -> []:
        
        _json = self._json_cache.get(f"{for_date.month}/{for_date.day}/{for_date.year}", None)
        
        if _json is None:
            client = Socrata(domain="data.cdc.gov", app_token="XuVpHSeARG3K6hAF3scIiqIx6",)
            _d = f"{for_date.year}-{for_date.month}-{for_date.day}"
            _json = client.get("8xkx-amqh", date=_d)
            self._json_cache.update({f"{for_date.month}/{for_date.day}/{for_date.year}": _json})

        return _json
        
    
    def _get_dataframe_for_date(self, for_date: datetime.date, data_type:str) -> pd.DataFrame:
        _json = self._get_json_data_for_date(for_date)
        _index_columns = ['fips', 'recip_county', 'recip_state']
        _date_column = 'date'

        try:
            # get data for date in dataframe format
            _df_date = pd.DataFrame(_json)

            # convert data types
            _df_date[data_type] = pd.to_numeric(_df_date[data_type])
            _df_date[_date_column] = pd.to_datetime(_df_date[_date_column])
            _df_date.fips = _df_date.fips.astype(str).str.pad(width=5, side='left', fillchar='0')

            # create dataframe with single measurement with date as a column
            _df_date = _df_date.pivot_table(index=_index_columns, columns=_date_column, values=data_type)

            # convert date column header
            _df_date.columns = [f"{c.month}/{c.day}/{c.year}" for c in _df_date.columns]

            return _df_date
        except:
            return None
    
    def base_date_formatted(self, delimiter:str = '/') -> str:
        if delimiter == '-':
            return f"{self._base_date.year}-{self._base_date.month}-{self._base_date.day}"
        
        return f"{self._base_date.month}/{self._base_date.day}/{self._base_date.year}"
    
    def _update_dataframe(self, data_type:str, verbose:bool):
        # get the existing dataframe from the _vax_dfs
        _df = self._vax_dfs.get(data_type, None)
        if _df is None:
            print(f"No data for '{data_type}'")
            return
        
        # add new day of data is warranted
        
        # determine what yesterday's date
        _yesterday = datetime.datetime.today() - datetime.timedelta(days=1)
        
        # convert df string dates to datetime and get the max
        _dates = [datetime.datetime.strptime(_d, "%m/%d/%Y") for _d in _df.columns]
        _max_date = max(_dates)
        
        if verbose:
            print(f"Updating '{data_type}' vax data ", end ="")
        while _max_date <= _yesterday:
            _new_date = _max_date + datetime.timedelta(days=1)
            
            _new_df = self._get_dataframe_for_date(_new_date, data_type)
            if _new_df is not None:
    #             _df = pd.concat([_df, _new_df], axis=1)
                _df = _df.join(_new_df)

            _max_date = _new_date
            if verbose:
                print(".", end="")
        
        # fill NaNs from left
        try:
            _df[self.base_date_formatted()].fillna(value=0, inplace=True)
            _df = _df.fillna(method='ffill', axis=1)
        except Exception as e:
            print(e)
            print (_df)
        
        self._vax_dfs.update({data_type: _df})
        _df.to_csv(os.path.join(FILE_PATH_VAXDATA, data_type))
        if verbose:
            print(f"'{data_type}' data updated!")
    
    
    ### RETRIEVE DATA
    
    # get df. checks for updated info first.
    def get_df(self, data_type:str, state:str=None, fips:str=None, detailed:bool=True, verbose=False, percentage=False):
        self._update_dataframe(data_type, verbose)
        _df = self._vax_dfs.get(data_type)
        
        # adjust scaling so pct is indecimal form and units are in 000's
        if data_type.endswith('_pct'):
            _df = _df * .01
        
        # get state totals
        if state is not None and detailed == False:
            rv = _df[_df.index.get_level_values('recip_state')==state].groupby(by='recip_state').sum()
            
            # for state percentage, calculate value
            if percentage is True:
                _state_pop = self._get_state_population(state)
                rv = rv / _state_pop
            
            return rv
        
        # get state with county detail
        if state is not None and detailed == True:
            return _df[_df.index.get_level_values('recip_state')==state] #.groupby(by='recip_state').sum()
        
        # get fips
        if fips is not None:
            return _df[_df.index.get_level_values('fips')==fips] #.groupby(by='recip_state').sum()
        
        # list all by state
        if detailed==False:
            return _df.groupby(by='recip_state').sum() #.reset_index()
        
        return _df
    
    def _get_population_df(self) -> pd.DataFrame:
        c = Census(os.getenv("CENSUS_API_KEY"), year=2010)
        p_df = pd.DataFrame(
            c.sf1.state_county(['NAME', 'P001001'], state_fips=Census.ALL, county_fips=Census.ALL))
        p_df.rename(
            columns={'P001001': 'population', 'state': 'state_fips', 'county': 'county_fips'}, inplace=True)
        p_df.population = p_df.population.astype(int)
        p_df['FIPS'] = p_df.state_fips + p_df.county_fips
        
        # add state abbr to df
        fips_lookup_dict = {x.fips: x.abbr for x in states.STATES}
        p_df['state_abbr'] = [fips_lookup_dict.get(s) for s in p_df.state_fips] 
                
        p_df = p_df.set_index('FIPS')

        return p_df
    
    def _get_fips_population(self, fips:str) -> int:
        # get a county population by its fips value
        rv = self._population_df[self._population_df.index==fips].population.values[0]
        return rv
    
    def _get_state_population(self, state:str) -> int:
        # get the population for a state by its abbreviation
        _pop = self._population_df[self._population_df['state_abbr']==state].population.sum()
    
        return _pop
    
    def _get_counties_by_state(self, state:str, data_type:str) -> pd.DataFrame:
        # get a df that includes the vaccinations by country for a given state and data_type
        rv = self.get_df(data_type=data_type)
        rv = rv[rv.index.get_level_values(2) == state]
        return rv
        
    
    
        
    

In [17]:
vd = Vax_Data(verbose=False)

In [14]:
vd._vax_dfs.keys()

dict_keys(['series_complete_pop_pct', 'series_complete_yes', 'series_complete_12plus', 'series_complete_12pluspop', 'series_complete_18plus', 'series_complete_18pluspop', 'series_complete_65plus', 'series_complete_65pluspop', 'completeness_pct', 'administered_dose1_recip', 'administered_dose1_pop_pct', 'administered_dose1_recip_12plus', 'administered_dose1_recip_12pluspop_pct', 'administered_dose1_recip_18plus', 'administered_dose1_recip_18pluspop_pct', 'administered_dose1_recip_65plus', 'administered_dose1_recip_65pluspop_pct'])

In [44]:
vd._get_counties_by_state('IA', 'series_complete_65plus')['7/9/2021'] # number of people

fips   recip_county        recip_state
19013  Black Hawk County   IA             19148.0
19017  Bremer County       IA              4276.0
19021  Buena Vista County  IA              2659.0
19041  Clay County         IA              2790.0
19043  Clayton County      IA              3212.0
19045  Clinton County      IA              7603.0
19047  Crawford County     IA              2410.0
19055  Delaware County     IA              2853.0
19059  Dickinson County    IA              3796.0
19085  Harrison County     IA              2284.0
19101  Jefferson County    IA              2923.0
19105  Jones County        IA              3620.0
19113  Linn County         IA             32193.0
19135  Monroe County       IA              1219.0
19139  Muscatine County    IA              6257.0
19141  O'Brien County      IA              2329.0
19145  Page County         IA              2665.0
19151  Pocahontas County   IA              1280.0
19159  Ringgold County     IA               953.0
19177  Van 

In [43]:
(vd._get_counties_by_state('IA', 'series_complete_65pluspop')/100)['7/9/2021'] # percent of people

fips   recip_county        recip_state
19013  Black Hawk County   IA             0.861
19017  Bremer County       IA             0.859
19021  Buena Vista County  IA             0.813
19041  Clay County         IA             0.824
19043  Clayton County      IA             0.752
19045  Clinton County      IA             0.822
19047  Crawford County     IA             0.798
19055  Delaware County     IA             0.848
19059  Dickinson County    IA             0.823
19085  Harrison County     IA             0.819
19101  Jefferson County    IA             0.674
19105  Jones County        IA             0.827
19113  Linn County         IA             0.874
19135  Monroe County       IA             0.779
19139  Muscatine County    IA             0.852
19141  O'Brien County      IA             0.820
19145  Page County         IA             0.764
19151  Pocahontas County   IA             0.825
19159  Ringgold County     IA             0.785
19177  Van Buren County    IA             0.736
1

In [48]:
(vd._get_counties_by_state('IA', 'series_complete_65plus')['7/9/2021']/(vd._get_counties_by_state('IA', 'series_complete_65pluspop')/100)['7/9/2021']).sum()

143394.02049366824

In [59]:
(vd._population_df[vd._population_df['state_abbr']=='IA'])

,NAME,population,state_fips,county_fips,state_abbr
FIPS,,,,,
19001,"Adair County, Iowa",7682,19,001,IA
19017,"Bremer County, Iowa",24276,19,017,IA
19005,"Allamakee County, Iowa",14330,19,005,IA
19003,"Adams County, Iowa",4029,19,003,IA
19013,"Black Hawk County, Iowa",131090,19,013,IA
19011,"Benton County, Iowa",26076,19,011,IA
19009,"Audubon County, Iowa",6119,19,009,IA
19007,"Appanoose County, Iowa",12887,19,007,IA
19015,"Boone County, Iowa",26306,19,015,IA


In [68]:
rv = vd.get_df(data_type='series_complete_yes', detailed=True)

In [69]:
rv[rv.index.get_level_values(1) == 'Scott County']

,,,12/13/2020,12/14/2020,12/15/2020,12/16/2020,12/17/2020,12/18/2020,12/19/2020,12/20/2020,12/21/2020,12/22/2020,12/23/2020,12/24/2020,12/25/2020,12/26/2020,12/27/2020,12/28/2020,12/29/2020,12/30/2020,12/31/2020,1/1/2021,1/2/2021,1/3/2021,1/4/2021,1/5/2021,1/6/2021,1/7/2021,1/8/2021,1/9/2021,1/10/2021,1/11/2021,1/12/2021,1/13/2021,1/14/2021,1/15/2021,1/16/2021,1/17/2021,1/18/2021,1/19/2021,1/20/2021,1/21/2021,1/22/2021,1/23/2021,1/24/2021,1/25/2021,1/26/2021,1/27/2021,1/28/2021,1/29/2021,1/30/2021,1/31/2021,2/1/2021,2/2/2021,2/3/2021,2/4/2021,2/5/2021,2/6/2021,2/7/2021,2/8/2021,2/9/2021,2/10/2021,2/11/2021,2/12/2021,2/13/2021,2/14/2021,2/15/2021,2/16/2021,2/17/2021,2/18/2021,2/19/2021,2/20/2021,2/21/2021,2/22/2021,2/23/2021,2/24/2021,2/25/2021,2/26/2021,2/27/2021,2/28/2021,3/1/2021,3/2/2021,3/3/2021,3/4/2021,3/5/2021,3/6/2021,3/7/2021,3/8/2021,3/9/2021,3/10/2021,3/11/2021,3/12/2021,3/13/2021,3/14/2021,3/15/2021,3/16/2021,3/17/2021,3/18/2021,3/19/2021,3/20/2021,3/21/2021,3/22/2021,3/23/2021,3/24/2021,3/25/2021,3/26/2021,3/27/2021,3/28/2021,3/29/2021,3/30/2021,3/31/2021,4/1/2021,4/2/2021,4/3/2021,4/4/2021,4/5/2021,4/6/2021,4/7/2021,4/8/2021,4/9/2021,4/10/2021,4/11/2021,4/12/2021,4/13/2021,4/14/2021,4/15/2021,4/16/2021,4/17/2021,4/18/2021,4/19/2021,4/20/2021,4/21/2021,4/22/2021,4/23/2021,4/24/2021,4/25/2021,4/26/2021,4/27/2021,4/28/2021,4/29/2021,4/30/2021,5/1/2021,5/2/2021,5/3/2021,5/4/2021,5/5/2021,5/6/2021,5/7/2021,5/8/2021,5/9/2021,5/10/2021,5/11/2021,5/12/2021,5/13/2021,5/14/2021,5/15/2021,5/16/2021,5/17/2021,5/18/2021,5/19/2021,5/20/2021,5/21/2021,5/22/2021,5/23/2021,5/24/2021,5/25/2021,5/26/2021,5/27/2021,5/28/2021,5/29/2021,5/30/2021,5/31/2021,6/1/2021,6/2/2021,6/3/2021,6/4/2021,6/5/2021,6/6/2021,6/7/2021,6/8/2021,6/9/2021,6/10/2021,6/11/2021,6/12/2021,6/13/2021,6/14/2021,6/15/2021,6/16/2021,6/17/2021,6/18/2021,6/19/2021,6/20/2021,6/21/2021,6/22/2021,6/23/2021,6/24/2021,6/25/2021,6/26/2021,6/27/2021,6/28/2021,6/29/2021,6/30/2021,7/1/2021,7/2/2021,7/3/2021,7/4/2021,7/5/2021,7/6/2021,7/7/2021,7/8/2021,7/9/2021
fips,recip_county,recip_state,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
20171,Scott County,KS,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,110.0,110.0,124.0,124.0,124.0,124.0,124.0,124.0,124.0,124.0,265.0,265.0,265.0,265.0,265.0,265.0,340.0,340.0,340.0,340.0,340.0,380.0,380.0,461.0,461.0,461.0,461.0,461.0,461.0,572.0,572.0,572.0,589.0,589.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,628.0,881.0,881.0,881.0,881.0,881.0,1000.0,1019.0,1019.0,1077.0,1077.0,1077.0,1170.0,1192.0,1192.0,1229.0,1237.0,1237.0,1237.0,1237.0,1237.0,1237.0,1237.0,1237.0,1237.0,1237.0,1237.0,1289.0,1289.0,1289.0,1289.0,1305.0,1305.0,1305.0,1305.0,1305.0,1305.0,1347.0,1347.0,1347.0,1347.0,1347.0,1371.0,1385.0,1387.0,1388.0,1391.0,1391.0,1391.0,1391.0,1418.0,1418.0,1418.0,1443.0,1443.0,1443.0,1443.0,1443.0,1443.0,1450.0,1450.0,1450.0,1458.0,1458.0,1458.0,1458.0,1469.0,1469.0,1469.0,1469.0,1469.0,1469.0,1469.0,1480.0,1483.0,1483.0,1484.0,1486.0,1489.0,1492.0,1492.0,1492.0,1492.0,1506.0,1507.0,1507.0,1507.0,1530.0,1530.0,1534.0,1535.0,1535.0,1535.0,1542.0,1544.0,1545.0,1545.0,1545.0,1547.0,1548.0,1558.0,1558.0,1558.0,1558.0,1558.0,1558.0
21209,Scott County,KY,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,289.0,289.0,289.0,289.0,289.0,289.0,289.0,289.0,655.0,655.0,655.0,764.0,764.0,764.0,764.0,764.0,1515.0,1613.0,1708.0,1708.0,1905.0,2222.0,2222.0,2222.0,2222.0,2222.0,2222.0,2222.0,2222.0,2816.0,2816.0,2934.0,2934.0,3262.0,3262.0,4387.0,4387.0,4774.0,4835.0

In [41]:
_pct = vd.get_df('series_complete_pop_pct', detailed=True)
_yes = vd.get_df('series_complete_yes', detailed=True)

In [42]:
_yes

,,,12/13/2020,12/14/2020,12/15/2020,12/16/2020,12/17/2020,12/18/2020,12/19/2020,12/20/2020,12/21/2020,12/22/2020,...,6/24/2021,6/25/2021,6/26/2021,6/27/2021,6/28/2021,6/29/2021,6/30/2021,7/1/2021,7/2/2021,7/3/2021
fips,recip_county,recip_state,,,,,,,,,,,,,,,,,,,,,
01001,Autauga County,AL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13565.0,13639.0,13639.0,13700.0,13700.0,13778.0,13778.0,13838.0,13838.0,13838.0
01009,Blount County,AL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,10576.0,10576.0,10576.0,10616.0,10616.0,10616.0,10616.0,10616.0,10616.0,10616.0
01027,Clay County,AL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,3390.0,3390.0,3423.0,3423.0,3423.0,3472.0,3472.0,3478.0,3478.0,3478.0
01039,Covington County,AL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,7950.0,7950.0,7967.0,7967.0,7967.0,7967.0,8017.0,8017.0,8017.0,8017.0
01065,Hale County,AL,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,5777.0,5777.0,5777.0,5826.0,5831.0,5831.0,5831.0,5831.0,5831.0,5831.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72143,Vega Alta Municipio,PR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13015.0,13015.0,13411.0,13411.0,13411.0,13648.0,13722.0,13722.0,15307.0,15307.0
72149,Villalba Municipio,PR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,13692.0,13692.0,13692.0,13692.0,14084.0,14084.0,14084.0,14084.0,14084.0,14084.0
72151,Yabucoa Municipio,PR,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,12417.0,12417.0,12417.0,12718.0,12718.0,12718.0,12718.0,14876.0,14876.0,14876.0


In [281]:
d = get_dataframe('deaths_US')

In [426]:
d['df'].columns

Index(['UID', 'iso2', 'iso3', 'code3', 'FIPS', 'County', 'Province_State',
       'Country_Region', 'Lat', 'Long_',
       ...
       '4/26/21', '4/27/21', '4/28/21', '4/29/21', '4/30/21', '5/1/21',
       'political_affiliation', 'County_State', 'population', 'geometry'],
      dtype='object', length=443)

In [276]:
# GET CSV DATA, CLEAN DATA, AND PLACE INTO DICTIONARY FOR EASY ACCESS
def get_dataframe(dataset, file_path: str = FILE_PATH) -> dict:
    """
    Support function that will retrieve a dictionary which includes a dataset in a dataframe format for simplified downstream sorting and filtering.

    :param dataset: The string name of the dataset for which data is retrieved ('recovered_global', 'deaths_global', 'deaths_US', 'confirmed_global', 'confirmed_US')
    :param file_path: (optional) The root filepath where the applicable data is stored.

    :return: Returns a dictionary with the dataframe along with applicable parameters useful in handling the dataset. The dictionary container the following key-values {'df': <dataframe>, 'attr_cols': <list of names of attribute columns>, 'date_cols_text': <list of date columns in text format>, 'date_cols_dates': <list of date columns in datetime format>}
    """
    file_names = {
        'recovered_global': 'time_series_covid19_recovered_global.csv',
        'deaths_global': 'time_series_covid19_deaths_global.csv',
        'deaths_US': 'time_series_covid19_deaths_US.csv',
        'confirmed_global': 'time_series_covid19_confirmed_global.csv',
        'confirmed_US': 'time_series_covid19_confirmed_US.csv'
    }

    file_name = file_names.get(dataset)

    def get_pol_aff(s):
        return political_affiliations.get(s.Province_State, 'na')

    def make_county_state(s):
        return f"{s.County}, {s.Province_State}"

    f = os.path.join(file_path, file_name)
    if not os.path.isfile(f):
        logging.error(f"{f} is not a file.")

    df = pd.DataFrame()
    if os.path.isfile(f):
        df = pd.read_csv(f)
        if "Province_State" in df.columns:
            # add political affiliation
            df['political_affiliation'] = df.apply(get_pol_aff, axis=1)

            if "Admin2" in df.columns:
                df = df.rename(columns={'Admin2': 'County'})
                df['County_State'] = df.apply(make_county_state, axis=1)

            # format FIPS
            df['FIPS'] = df['UID'].astype(str).apply(lambda x: x[3:])

            # add population by county
            c = Census(os.getenv("CENSUS_API_KEY"), year=2010)
            county_population = pd.DataFrame(
                c.sf1.state_county(['NAME', 'P001001'], state_fips=Census.ALL, county_fips=Census.ALL))
            county_population.rename(
                columns={'P001001': 'population', 'state': 'state_fips', 'county': 'county_fips'}, inplace=True)
            county_population.population = county_population.population.astype(int)
            county_population['FIPS'] = county_population.state_fips + county_population.county_fips
            df = df.merge(county_population[['FIPS', 'population']], on=['FIPS'])

    else:
        if not os.path.isdir(file_path):
            logging.error(f"NOT A PATH: '{file_path}''")
        else:
            logging.error(f"NOT A FILE: '{file_name}''")

    # delete jan and feb
    cols_to_remove = []
    for c in df.columns:
        spl = c.split('/')
        if len(spl) == 3 and spl[2] == '20':
            if spl[0] == '1' or spl[0] == '2':
                cols_to_remove.append(c)
    df.drop(columns=cols_to_remove, inplace=True)

    # remove US territories
    if "Province_State" in df.columns:
        df = df[~df.Province_State.isin(territories)]

    # convert to GeoPandas if there are lat and lon coordinates
    if "Lat" in df.columns and "Long_" in df.columns:
        df = gpd.GeoDataFrame(df, geometry=gpd.points_from_xy(df.Long_, df.Lat))

    attr_cols, date_cols_text, date_cols_dates = get_column_groups(df)

    return {'df': df, 'attr_cols': attr_cols, 'date_cols_text': date_cols_text, 'date_cols_dates': date_cols_dates}

def get_column_groups(df):
    """
    Returns a tuple of 3 lists that include the columns of a dataframe split into 3 groups. 'Attribute Columns' are non-date columns in a dataframe,  'Date Columns Text' are date columns in text format and 'Date Columns Date' are date columns in datetime format.

    :param df:  The dataframe that will be evaluated and for which the column groups are returned.

    :return: A tuple of 3 lists: the attribute columns (all columns that are not dates), date columns in text format and the date columns is datetime format.
    """
    date_cols_dates = pd.to_datetime(df.columns, errors='coerce')
    date_cols_tf = [not pd.isnull(c) for c in date_cols_dates]
    date_cols_dates = date_cols_dates[date_cols_tf]
    date_cols_text = df.columns[date_cols_tf]
    attr_cols = df.columns[[not c for c in date_cols_tf]]

    return attr_cols, date_cols_text, date_cols_dates


# SUPPORT FUNCTIONS
# group by state
def get_df_by_state(_df) -> pd.DataFrame:
    """
    Support function that will return a dataframe grouped by 'Province State' where values are summed.

    :param _df:  The dataframe that will be grouped.

    :return: The grouped dataframe
    """
    return _df.groupby(by='Province_State').sum().reset_index()


# group by county in a state
def get_df_by_counties(_df, state) -> pd.DataFrame:
    """
    A support function that will return the entries in a provided dataframe that match the provided state.  The state value provided is not case sensitive.

    :param _df:  The initial dataframe that will be filtered.
    :param state: The state string name that will be used to filter the dataframe.

    :return: A dataframe that includes the entries of the provided dataframe that only include the provided state.
    """
    return _df[_df.Province_State.str.lower() == state.lower()]


def get_rankings(_df, top_n=None) -> list:
    """
    Support function that will return a list of the top n listings in a dataframe based on the most recent date.  Results are listing in descending order.  If no top_n value is provided, all items in the dataframe are sorted.

    :param _df:  The dataframe that will be filtered and sorted.
    :param top_n: (optional) An integer value that will restrict the returned list to the top n number of listings.

    :return: A list of top_n indexes from a dataframe
    """
    attr_cols, date_cols_text, date_cols_dates = get_column_groups(_df)
    a_cols = [c for c in _df.columns if c in attr_cols]

    rv_df = _df[date_cols_text].rank(ascending=False)
    rv_df = pd.concat([_df.loc[rv_df.index][a_cols], rv_df], axis=1)

    if top_n is not None:
        return rv_df.nsmallest(top_n, date_cols_text[-1]).sort_values(by=date_cols_text[-1], ascending=True)

    return rv_df.sort_values(by=date_cols_text[-1])


# get data per day
def get_by_day(_df):
    """
    Return a dataframe that contains cumulative values in date columns in a daily value format.

    :param _df:  A dataframe where the values in each date column are cumulative values.

    :return: The supplied dataframe with daily data instead of cumulative data.
    """
    _, date_cols_text, _ = get_column_groups(_df)
    daily = _df[date_cols_text] - _df[date_cols_text].shift(axis=1)
    attr_cols = set(_df.columns) - set(date_cols_text)
    daily = pd.concat([_df[attr_cols], daily], axis=1)

    return daily


def get_daily_growth_rate(_df):
    """
    Return a dataframe that includes values that represent the change rate vs the prior day for all values in date columns.

    :param _df:  A dataframe of cumulative values in date columns

    :return: A dataframe with the same columns and rows as the original dataframe, except the values are daily change rates rather than cumulative absolute values.
    """
    _, date_cols_text, _ = get_column_groups(_df)
    _df = get_by_day(_df)
    daily_rate = _df[date_cols_text] / _df[date_cols_text].shift(axis=1)
    attr_cols = set(_df.columns) - set(date_cols_text)
    daily_rate = pd.concat([_df[attr_cols], daily_rate], axis=1)
    daily_rate = daily_rate.fillna(1)
    return daily_rate




In [277]:
from bokeh.embed import json_item
from bokeh.plotting import figure
from bokeh.models import FactorRange, ColumnDataSource, HoverTool, NumeralTickFormatter
from bokeh.models.callbacks import CustomJS

from django.http import JsonResponse

# from .helpers import *


def _get_plot_data(data_type, frequency, state, county):
    # set dataframes
    if data_type == 'infections':
        df_dict = get_dataframe('confirmed_US')
    else:
        df_dict = get_dataframe('deaths_US')

    # _, date_cols_text, date_cols_dates = get_column_groups(df)

    df = df_dict['df']
    date_cols_text = df_dict['date_cols_text']
    date_cols_dates = df_dict['date_cols_dates']

    if frequency == 'daily':
        all_data = get_by_day(df)
    else:
        all_data = df.copy()

    if state == 'United States':
        plot_data = all_data.sum()[date_cols_text].values
    else:
        if county == 'All':
            plot_data = all_data[all_data.Province_State == state].sum()[date_cols_text].values
        else:
            plot_data = all_data[(all_data.Province_State == state) & (all_data.County == county)].sum()[
                date_cols_text].values

    # setup x axis groupings
    factors = [(str(c.year), c.month_name(), str(c.day)) for c in date_cols_dates]

    return plot_data, factors


def plot_state_chart(request, state="United States", county='All', frequency='daily', data_type='infections', rolling_window=14):
    """
    Plots the values for a selected state, county or the entire United States.

    :param request:  The HTML request that should include values for 'state', 'county', 'frequency', 'data_type' and 'rolling_window'.  See the parameter descriptions below for contraints and defaults for each parameter.
    :param state: (optional) Default 'Massachusetts'.  The name of the state to plot.
    :param county: (optional) Default 'All'.  The county to plot or 'All' counties of a state of value is 'All'.
    :param frequency: (optional) Default 'daily'.  Whether to plot daily or cumulative values.
    :param data_type: (optional) Default 'infections'.  Plot 'infections' data or 'deaths' data.
    :param rolling_window: (optional) Default 14.  The number of days to use when drawing the daily average line in the plot.

    :return: A Bokeh JSON formatted plot that can be handled by JavaScript for HTML presentation.
    """
    if request is not None:
        state = request.GET.get('state', 'United States')
        county = request.GET.get('county', 'All')
        frequency = request.GET.get('frequency', 'daily')
        data_type = request.GET.get('data_type', 'infections').lower()
        rolling_window = int(request.GET.get('rolling_window', 14))

    state = ' '.join([word.capitalize() for word in state.split(' ')])
    county = county.capitalize()

    plot_data, factors = _get_plot_data(data_type, frequency, state, county)

    # # setup x axis groupings
    # factors = [(c.month_name(), str(c.day)) for c in DATE_COLS_DATES]

    # setup Hover tool
    hover = HoverTool()
    hover.tooltips = [
        ("Date", "@date"),
        (data_type.capitalize(), "@val{0,0}"),
        (f"{rolling_window}-day Avg", "@rolling_avg{0,0.0}")
    ]

    # setup figure
    p = figure(x_range=FactorRange(*factors), sizing_mode='stretch_both',  # plot_height=500, plot_width=900,
               y_axis_type='linear', y_axis_label=data_type, output_backend="webgl",
               toolbar_location=None, tools=[hover],
               title=f"{state} New {data_type.capitalize()}{' by Day' if frequency == 'daily' else ''}")
    p.title.text_font_size = '12pt'
    p.yaxis.formatter = NumeralTickFormatter(format="0,000")

    source = ColumnDataSource(
        data=dict(date=factors, val=plot_data, rolling_avg=pd.Series(plot_data).rolling(rolling_window).mean().values))

    b = p.vbar(x='date', top='val', source=source, color='red', width=.5)

    if frequency == 'daily':
        l = p.line(x='date', y='rolling_avg', source=source, color='black', width=3, legend_label=f"{rolling_window}-Day Rolling Average")
        p.legend.location = 'top_left'

    p.xaxis.major_label_orientation = 1
    p.xaxis.group_text_font_size = "10pt"  # months size
    p.xaxis.major_label_text_font_size = "6pt"  # date size
    p.yaxis.major_label_orientation = 1
    p.xgrid.grid_line_color = None

    callback = CustomJS(args=dict(source=source), code="""

        // JavaScript code goes here

        var updated_data; 

        // the model that triggered the callback is cb_obj:
        fetch(cb_obj.value)
        .then( response = return response.json() )
        .then( x => udpated_data = x ) 

        // models passed as args are automagically available
        source.data = updated_data;

        """)
    
    return p
#     return JsonResponse(json_item(p))

In [278]:
from bokeh.plotting import show
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [279]:
_p = plot_state_chart(None)

In [280]:
show(_p)

In [25]:
plot_data, factors = _get_plot_data('infections', "daily", "United States", "All")

In [28]:
len(plot_data)

413

In [32]:
factors

[('March', '1', '2020'),
 ('March', '2', '2020'),
 ('March', '3', '2020'),
 ('March', '4', '2020'),
 ('March', '5', '2020'),
 ('March', '6', '2020'),
 ('March', '7', '2020'),
 ('March', '8', '2020'),
 ('March', '9', '2020'),
 ('March', '10', '2020'),
 ('March', '11', '2020'),
 ('March', '12', '2020'),
 ('March', '13', '2020'),
 ('March', '14', '2020'),
 ('March', '15', '2020'),
 ('March', '16', '2020'),
 ('March', '17', '2020'),
 ('March', '18', '2020'),
 ('March', '19', '2020'),
 ('March', '20', '2020'),
 ('March', '21', '2020'),
 ('March', '22', '2020'),
 ('March', '23', '2020'),
 ('March', '24', '2020'),
 ('March', '25', '2020'),
 ('March', '26', '2020'),
 ('March', '27', '2020'),
 ('March', '28', '2020'),
 ('March', '29', '2020'),
 ('March', '30', '2020'),
 ('March', '31', '2020'),
 ('April', '1', '2020'),
 ('April', '2', '2020'),
 ('April', '3', '2020'),
 ('April', '4', '2020'),
 ('April', '5', '2020'),
 ('April', '6', '2020'),
 ('April', '7', '2020'),
 ('April', '8', '2020'),
 ('